![](Compliance.png)

### Libraries
Import all required libraries for data analysis and visualization.

In [ ]:
import kaggle
import pandas as pd
import os
import streamlit as st
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go

kaggle.api.authenticate()
dataset_name = "atharvasoundankar/big-4-financial-risk-insights-2020-2025"
kaggle.api.dataset_download_files(dataset_name, path=".", unzip=True)
csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
if csv_files:
    df = pd.read_csv(csv_files[0])
    print(df.head())

Dataset URL: https://www.kaggle.com/datasets/atharvasoundankar/big-4-financial-risk-insights-2020-2025
   Year Firm_Name  Total_Audit_Engagements  High_Risk_Cases  \
0  2020       PwC                     2829               51   
1  2022  Deloitte                     3589              185   
2  2020       PwC                     2438              212   
3  2021       PwC                     2646              397   
4  2020       PwC                     2680              216   

   Compliance_Violations  Fraud_Cases_Detected Industry_Affected  \
0                    123                    39        Healthcare   
1                     30                    60        Healthcare   
2                    124                    97        Healthcare   
3                     55                    97        Healthcare   
4                     99                    46        Healthcare   

   Total_Revenue_Impact AI_Used_for_Auditing  Employee_Workload  \
0                114.24                   

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year                       100 non-null    int64  
 1   Firm_Name                  100 non-null    object 
 2   Total_Audit_Engagements    100 non-null    int64  
 3   High_Risk_Cases            100 non-null    int64  
 4   Compliance_Violations      100 non-null    int64  
 5   Fraud_Cases_Detected       100 non-null    int64  
 6   Industry_Affected          100 non-null    object 
 7   Total_Revenue_Impact       100 non-null    float64
 8   AI_Used_for_Auditing       100 non-null    object 
 9   Employee_Workload          100 non-null    int64  
 10  Audit_Effectiveness_Score  100 non-null    float64
 11  Client_Satisfaction_Score  100 non-null    float64
dtypes: float64(3), int64(6), object(3)
memory usage: 9.5+ KB
None


# Clean and Categorize

In [ ]:
df.drop_duplicates()
duplicate_count = df.duplicated().sum()
nan_count = df.isna().sum().sum()
print(f'Duplicated rows: {duplicate_count}')
print(f'Missing values: {nan_count}')

Duplicated rows: 0 
Missing values: 0 


## 1. Compliance Trend Over Time

In [ ]:
compliance_trend = df.groupby("Year")[["Total_Audit_Engagements", "High_Risk_Cases", "Compliance_Violations"]].sum().reset_index()
display(compliance_trend)

,Total_Audit_Engagements,High_Risk_Cases,Compliance_Violations
Year,,,
2020,55139,6063,2441
2021,51888,4684,1562
2022,43667,4195,1556
2023,35752,3006,1454
2024,43753,5513,1529
2025,48253,4312,2006


In [ ]:
# Totals of compliance violations and the firms that found them
firm_compliance = df.groupby("Firm_Name")[["Total_Audit_Engagements"]].sum().reset_index()
display(firm_compliance)

,Firm_Name,Total_Audit_Engagements
0,Deloitte,71631
1,Ernst & Young,59730
2,KPMG,67155
3,PwC,79936


## 2. Fraud and Revenue Impact

In [ ]:
firm_fraud = df.groupby("Firm_Name")[["Fraud_Cases_Detected"]].sum().reset_index()
display(firm_fraud)

,Firm_Name,Fraud_Cases_Detected
0,Deloitte,1481
1,Ernst & Young,1031
2,KPMG,1319
3,PwC,1439


In [ ]:
sector_fraud = df.groupby("Industry_Affected")[["Fraud_Cases_Detected"]].sum().reset_index()
display(sector_fraud)

,Industry_Affected,Fraud_Cases_Detected
0,Finance,1101
1,Healthcare,1410
2,Retail,1365
3,Tech,1394


In [ ]:
revenue_loss = df.groupby("Year")[["Total_Revenue_Impact"]].sum().reset_index()
display(revenue_loss)

,Year,Total_Revenue_Impact
0,2020,4509.59
1,2021,6094.55
2,2022,3325.64
3,2023,3325.98
4,2024,4629.88
5,2025,5368.26


### Audit Frequency and Employee Workload

In [ ]:
# Audits frequency over the years
audit_frequency = df.groupby("Year")[["Total_Audit_Engagements"]].sum().reset_index()
display(audit_frequency)

# Employee workload over the years
employee_workload = df.groupby("Year")[["Employee_Workload"]].mean().reset_index()
display(employee_workload)

# Shows a steady increase in audits

Audit Frequency Over The Years


,Year,Total_Audit_Engagements
0,2020,55139
1,2021,51888
2,2022,43667
3,2023,35752
4,2024,43753
5,2025,48253


Avg. Employee Workload Over The Years


,Year,Employee_Workload
0,2020,60.095238
1,2021,61.157895
2,2022,62.333333
3,2023,59.769231
4,2024,59.312500
5,2025,58.750000


In [ ]:
workload_firm = df.groupby('Firm_Name')[['Employee_Workload']].mean().reset_index()
display(workload_firm)

,Firm_Name,Employee_Workload
0,Deloitte,62.433333
1,Ernst & Young,60.913043
2,KPMG,59.227273
3,PwC,57.920000


## 3. AI Audits: Success or Failures?
Summary of AI usage in auditing and its impact on compliance, fraud, and revenue.

In [ ]:
ai_yes = df[df['AI_Used_for_Auditing'] == 'Yes']
ai_year_yes = ai_yes.groupby('Year').size().reset_index(name='AI_Used_Audit_Count')
display(ai_year_yes)


,Year,AI_Used_Audit_Count
0,2020,7
1,2021,9
2,2022,7
3,2023,6
4,2024,6
5,2025,10


In [ ]:
# Compliance violations and AI usage by firm
bar_yes = ai_yes.groupby('Firm_Name')['Compliance_Violations'].sum().reset_index()
print('AI USED')
display(bar_yes)

ai_no = df[df['AI_Used_for_Auditing'] == 'No']
bar_no = ai_no.groupby('Firm_Name')['Compliance_Violations'].sum().reset_index()
print('AI NOT USED')
display(bar_no)


 AI USED 
        Firm_Name  Compliance_Violations
0       Deloitte                    894
1  Ernst & Young                   1124
2           KPMG                    801
3            PwC                   1607

 AI NOT USED 
        Firm_Name  Compliance_Violations
0       Deloitte                   2233
1  Ernst & Young                   1600
2           KPMG                   1269
3            PwC                   1020


In [ ]:
# AI usage in auditing and its impact on revenue loss
ai_revenue_summary = df.groupby('AI_Used_for_Auditing')['Total_Revenue_Impact'].mean().reset_index()
display(ai_revenue_summary)

,AI_Used_for_Auditing,Total_Revenue_Impact
0,No,292.984182
1,Yes,247.550444


## 4. Employee Workload by AI Usage
This table shows the average employee workload for audits where AI was and was not used.

In [ ]:
workload_ai = df.groupby('AI_Used_for_Auditing')[['Employee_Workload']].mean().reset_index()
display(workload_ai)

,AI_Used_for_Auditing,Employee_Workload
0,No,61.036364
1,Yes,59.288889


## 5. Employee Workload by Firm and AI Usage (2025)
This table shows the average employee workload for each firm, split by whether AI was used for auditing, for the year 2025 only.

In [88]:
# Average Employee Workload by Firm and AI Usage for 2025 only
workload_firm_ai_2025 = df[df['Year'] == 2025].groupby(['Firm_Name', 'AI_Used_for_Auditing'])[['Employee_Workload']].mean().reset_index()
display(workload_firm_ai_2025)

,Firm_Name,AI_Used_for_Auditing,Employee_Workload
0,Deloitte,No,69.000000
1,Deloitte,Yes,60.000000
2,Ernst & Young,No,56.500000
3,Ernst & Young,Yes,61.000000
4,KPMG,No,78.000000
5,KPMG,Yes,40.000000
6,PwC,No,45.000000
7,PwC,Yes,54.666667


In [90]:
workload_firm_ai = df.groupby(['Firm_Name', 'AI_Used_for_Auditing'])[['Employee_Workload']].mean().reset_index()
display(workload_firm_ai)

,Firm_Name,AI_Used_for_Auditing,Employee_Workload
0,Deloitte,No,64.523810
1,Deloitte,Yes,57.555556
2,Ernst & Young,No,57.000000
3,Ernst & Young,Yes,66.000000
4,KPMG,No,63.090909
5,KPMG,Yes,55.363636
6,PwC,No,56.700000
7,PwC,Yes,58.733333
